<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>

# 二维码动作  QRCode Movement

In [ ]:
# 中文开关，默认为英文 Chinese switch. The default value is English
g_ENABLE_CHINESE = False

Name_widgets = {
    'Start': ("Start", "开始"),
    'Close_Camera': ("Close_Camera", "关闭摄像头")
}

In [ ]:
# 显示摄像头组件 Camera display widgets
import ipywidgets.widgets as widgets
from IPython.display import display

import cv2
import time
import numpy as np


import threading


# 二维码解析库 QRCode library
import pyzbar.pyzbar as pyzbar
from PIL import Image

In [ ]:
#bgr8转jpeg格式
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
from SunriseRobotLib import Mipi_Camera

img_width=320
img_height=240

g_camera = Mipi_Camera(img_width, img_height)

if g_camera.isOpened():
    print("Open Camera OK")
else:
    print("Fail To Open Camera")


In [ ]:
g_stop_program = False
g_start_function = False
g_car_runtime = -1


In [ ]:
# image_widget = widgets.Image(format='jpeg', width=320, height=240)
# 摄像头显示控件  Camera display widgets
DISPLAY_WIDTH = img_width
DISPLAY_HEIGHT = img_height
image_widget = widgets.Image(format='jpeg', width=DISPLAY_WIDTH, height=DISPLAY_HEIGHT)

In [ ]:
# 开始 Start
Button_Start = widgets.Button(        
    description=Name_widgets['Start'][g_ENABLE_CHINESE],        
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

# 关闭摄像头 Close_Camera
Button_Close_Camera = widgets.Button(
    value=False,  
    description=Name_widgets['Close_Camera'][g_ENABLE_CHINESE],      
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

In [ ]:
# 按键按下事件处理   Key press event processing
def on_button_clicked(b):
    global g_stop_program
    b.icon = 'check'
    with output:
        print("Button clicked:", b.description)
    if b.description == Name_widgets['Close_Camera'][g_ENABLE_CHINESE]:
        # 停止线程，释放摄像头  Stop the thread and release the camera
        Button_Start.icon = 'uncheck'
        g_stop_program = True
        time.sleep(.1)
        g_camera.release()
        b.icon = 'uncheck'
        

# 功能按钮按下事件处理 Function button press event handling
def on_button_function_clicked(b):
    global g_start_function
    if b.description == Name_widgets['Start'][g_ENABLE_CHINESE]:
        if b.icon == 'uncheck':
            b.icon = 'check'
            g_start_function = True
        else:
            b.icon = 'uncheck'
            g_car_runtime = -1
            g_start_function = False
            time.sleep(.1)
            g_car.set_car_run(7, 0)
    with output:
        print("Button clicked:", b.description, g_start_function)

In [ ]:
# 关联按键事件回调 Button event callbacks
Button_Close_Camera.on_click(on_button_clicked)
Button_Start.on_click(on_button_function_clicked)

In [ ]:
from SunriseRobotLib import SunriseRobot
g_car = SunriseRobot()
car_speed = 0.3

In [ ]:

def car_control(info):
    # print(info)
    global g_car_runtime
    if g_car_runtime >= 0:
        return
    g_car_runtime = 100
    if info == "forward": # 前进
        g_car.set_car_run(1, car_speed*100)        
    elif info == "back": # 后退
        g_car.set_car_run(2, car_speed*100)       
    elif info == "left": # 左平移
        g_car.set_car_run(3, car_speed*100)
    elif info == "right": # 右平移
        g_car.set_car_run(1, car_speed*100)
    elif info == "turnleft": # 左旋转
        g_car.set_car_run(5, car_speed*100)
    elif info == "turnright": # 右旋转
        g_car.set_car_run(6, car_speed*100) 
    elif info == "stop": # 停车
        g_car.set_car_run(7, 0)
        g_car_runtime = -1

def detect_qrcode(image):
    # 转为灰度图像
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    barcodes = pyzbar.decode(gray)
    for barcode in barcodes:
        # 提取二维码的数据和边界框的位置 The data of the QR code and the position of the bounding box are extracted
        (x, y, w, h) = barcode.rect
        barcodeData = barcode.data.decode("utf-8")
        barcodeType = barcode.type
        # print("[INFO] Found {} barcode: {}".format(barcodeType, barcodeData))
        car_control(barcodeData)
        return barcodeData, (x, y, w, h)
    return None, (0, 0, 0, 0)

In [ ]:
def task_timeout():
    global g_car_runtime
    while True:
        if g_car_runtime > 0:
            print("g_car_runtime:", g_car_runtime)
            g_car_runtime = g_car_runtime - 1
        elif g_car_runtime == 0:
            print("auto stop")
            g_car.set_car_run(7, 0)
            g_car_runtime = -1
        time.sleep(.01)

# 图像处理任务 Image processing tasks
def task_processing():
    global g_stop_program, g_start_function
    t_start = time.time()
    m_fps = 0
    fps = 0
    while g_camera.isOpened():
        if g_stop_program:
            break
        ret, frame = g_camera.read()
        
        if g_start_function:
            payload, (x, y, w, h) = detect_qrcode(frame.copy())
            if payload != None:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 225, 255), 2)
                cv2.putText(frame, payload.upper(), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 225, 255), 1)
            cv2.putText(frame, "START " + str(int(fps)), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
            # time.sleep(.010)
            
        m_fps = m_fps + 1
        fps = m_fps / (time.time() - t_start)
        if (time.time() - t_start) >= 2:
            m_fps = fps
            t_start = time.time() - 1
        if not g_start_function:
            cv2.putText(frame, "FPS " + str(int(fps)), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
        
        # 图像传输给显示组件 The image is transmitted to the display component
        image_widget.value = bgr8_to_jpeg(frame)


In [ ]:
# 启动摄像头显示任务  Start the camera display task
thread1 = threading.Thread(target=task_processing)
thread1.setDaemon(True)
thread1.start()

thread2 = threading.Thread(target=task_timeout)
thread2.setDaemon(True)
thread2.start()

# 创建一个横向的盒子容器，以便将图像小部件相邻放置
# create a horizontal box container to place the image widget next to eachother
image_container = widgets.HBox([image_widget])
button_group = widgets.HBox([Button_Start, Button_Close_Camera])
output = widgets.Output()
box_display = widgets.VBox([image_container, button_group, output])
display(box_display)